In [4]:
import pandas as pd
import numpy as np

<img src="./ex1.JPG">

First of all I import csv files extracted from the excel sheet.

In [5]:
df_cc = pd.read_csv("./SONY_Exercise_1 - Customer claim.csv", index_col = 'CustomerNR' )
df_sap = pd.read_csv("./SONY_Exercise_1 - SAP.csv",index_col = 'Customer nr.')

After that, we define a function in order to check discrepancies.

In [6]:
def compensation_test(customer_claim,sap):
    customer_claims_ids = customer_claim.groupby(customer_claim.index)['QTY to compensate','End stock/Sellin'].sum()
    sap_ids = sap.groupby(sap.index)['Processed quantity'].sum()
    not_compensated = [(sap_id,"not_compensated") for sap_id in sap_ids.index if sap_id not in customer_claims_ids.index ]
    overcompensated =  [(customer_claims_id,'overcompensated') for customer_claims_id in customer_claims_ids.index if customer_claims_id not in sap_ids.index]
    wrong_claims = [(customer_claim_id,'wrong_claim') for customer_claim_id in customer_claims_ids.index if (customer_claims_ids.loc[customer_claim_id]['QTY to compensate'] - sap_ids.loc[customer_claim_id] != 0)]
    suspicious_compensations =  not_compensated + overcompensated + wrong_claims
    return  suspicious_compensations

In [7]:
compensation_test(df_cc,df_sap)

[(271500357, 'not_compensated'), (272500650, 'wrong_claim')]

Two discrepancies:
- 271500357 doesn't exist on customer claim sheet.
- 272500650 Claimed one unit more than corresponding on SAP registry.

<img src="./ex2.JPG">

First of all I import csv files extracted from the excel sheet.

In [8]:
df_ac = pd.read_csv("./SONY_Exercise_2 - Amounts to be compensated.csv",index_col = 'Model name' )
df_bq = pd.read_csv("./SONY_Exercise_2 - Billing qtys.csv",index_col = 'Material')

I define a function in order to calculate the costs.

In [9]:
def calculate_total_amount(amount_compensated,billing_quantitys):
    billing_quantitys['amount'] = [ amount_compensated.loc[index]['Amount/PC (CZK)']*qty for (index,qty) in zip(billing_quantitys.index,billing_quantitys['Bill qty'])]
    grouped_billing_quantitys = billing_quantitys.groupby(billing_quantitys.Name)['amount'].sum()
    return grouped_billing_quantitys,grouped_billing_quantitys.sum()

In [10]:
calculate_total_amount(df_ac,df_bq)

(Name
 AT COMPUTERS A.S.               8852
 Aaron shop s.r.o.              18620
 BER-KR SERVICES S.R.O.          4767
 CEWE COLOR a.s.                15820
 ELLEX GK S.R.O.                 5497
 Elvia pro, spol. s r.o.        16348
 MEGAPIXEL S.R.O.               20732
 Milan Skoda-FOTO               19556
 SWS A.S.                        8104
 VIDEO klinik, spol. s r. o.     1424
 Name: amount, dtype: int64, 119720)

Total costs is 119720

<img src="./ex3.JPG">

First of all I import csv file extracted from the excel sheet.

In [11]:
df_fy = pd.read_csv("./SONY_Exercise_3 - FY13 Sept-March DI.csv",index_col = 'Bill.Doc.')

I parse 'bill date' colum to datetime type.

In [ ]:
df_fy['date'] = pd.to_datetime(df_fy['Bill. Date'])

now we calculate the amount of bills on 2013-12-20

In [ ]:
ondate = df_fy.loc[df_fy['date'] == '2013-12-20']

In [ ]:
ondate

481 invoices were created on 20th of December of 2013

In [ ]:
krakow_invoices = df_fy.loc[df_fy['Name 1'] == 'Neonet S.A. Krakow']

In [ ]:
krakow_invoices

17 invoices were created for Neonet S.A. Krakow

Now we select Billed quantities greather than 60

In [ ]:
filtered = df_fy.loc[df_fy['Billed qty'] > 60][['Billed qty','  Cond.value']]

Cond.value column have an invalid format for numbers, so we must transform them.

In [ ]:
filtered['  Cond.value'] = [value.replace('.','')for value in filtered['  Cond.value']]
filtered['  Cond.value'] = [value.replace(',','.')for value in filtered['  Cond.value']]
filtered['  Cond.value'] = [float(value) for value in filtered['  Cond.value']]

In [ ]:
filtered


In [ ]:
filtered.sum()

- Total amount of bills is  5385.0
- Total amount of conditional value is 406696.8